<a href="https://colab.research.google.com/github/targoons/ML/blob/master/MLAssignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MsCA 31009 - Machine Learning and Predictive Analytics

## Assignment 3 - PCA & Logistic Regression

**Targoon Siripanichpong**

## Import files and libraries.

In [0]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, scale
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from scipy import stats
import math
import tensorflow as tf

**Mount Google Drive.**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**Read bottle.csv file.**

In [0]:
mypath = '/content/gdrive/My Drive/UChicago/Machine Learning/Data/Raw'
import os
os.listdir(mypath)

['ProviderInfo.csv', 'bottle.csv']

In [0]:
Provider_Info = pd.read_csv('/content/gdrive/My Drive/UChicago/Machine Learning/Data/Raw/ProviderInfo.csv')

## Data Exploration and Pre-Processing

In [0]:
Provider_Info.head()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,OWNERSHIP,...,CYCLE_2_REVISIT_SCORE,CYCLE_2_TOTAL_SCORE,WEIGHTED_ALL_CYCLES_SCORE,INCIDENT_CNT,CMPLNT_CNT,FINE_CNT,FINE_TOT,PAYDEN_CNT,TOT_PENLTY_CNT,FILEDATE
0,15009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653.0,2.563324e+09,290.0,Franklin,For profit - Individual,...,0,8,3.2,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01
1,15010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150.0,2.562496e+09,600.0,Talladega,For profit - Corporation,...,0,16,28,0.0,0.0,1.0,15259.0,1.0,2.0,2018-07-01
2,15012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768.0,2.562184e+09,350.0,Jackson,Government - County,...,0,12,31.2,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01
3,15014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206.0,2.058330e+09,360.0,Jefferson,For profit - Corporation,...,0,24,33.6,0.0,1.0,0.0,0.0,0.0,0.0,2018-07-01
4,15015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111.0,2.054776e+09,360.0,Jefferson,For profit - Corporation,...,0,24,19.2,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01


In [0]:
Provider_Info.shape

(15617, 78)

In [0]:
Provider_Info.describe()

,ZIP,PHONE,COUNTY_SSA,BEDCERT,RESTOT,OVERALL_RATING,SURVEY_RATING,QUALITY_RATING,STAFFING_RATING,RN_STAFFING_RATING,...,ADJ_AIDE,ADJ_LPN,ADJ_RN,ADJ_TOTAL,INCIDENT_CNT,CMPLNT_CNT,FINE_CNT,FINE_TOT,PAYDEN_CNT,TOT_PENLTY_CNT
count,15616.000000,1.561600e+04,15616.000000,15616.000000,15105.000000,15436.000000,15436.000000,15403.000000,15268.000000,15268.000000,...,14560.000000,14560.000000,14560.000000,14560.000000,15616.000000,15616.000000,15616.000000,1.561600e+04,15616.000000,15616.000000
mean,50352.172323,5.864225e+09,389.542328,106.119813,86.558782,3.332923,2.818412,3.981952,3.065431,3.264344,...,2.303975,0.880558,0.679904,3.855575,1.194032,4.198258,0.430712,1.458445e+04,0.098105,0.528817
std,26906.692569,2.330217e+09,273.754237,60.789892,52.999889,1.395167,1.289865,1.228714,1.261812,1.337252,...,0.559962,0.341518,0.441747,0.858477,3.093278,7.384479,0.773936,5.500255e+04,0.355164,0.966987
min,660.000000,2.012270e+09,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,1.230690,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,29801.750000,3.866777e+09,150.000000,64.000000,51.700000,2.000000,2.000000,3.000000,2.000000,2.000000,...,1.931877,0.672057,0.398147,3.312228,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,49008.000000,6.056590e+09,360.000000,99.000000,78.700000,4.000000,3.000000,4.000000,3.000000,3.000000,...,2.214275,0.875080,0.587110,3.726035,0.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,72833.250000,8.014871e+09,590.000000,127.000000,108.700000,5.000000,4.000000,5.000000,4.000000,4.000000,...,2.582335,1.074725,0.854212,4.229495,1.000000,5.000000,1.000000,5.336250e+03,0.000000,1.000000
max,99929.000000,1.000000e+10,996.000000,1389.000000,754.900000,5.000000,5.000000,5.000000,5.000000,5.000000,...,6.593350,5.560940,7.775820,11.726110,126.000000,135.000000,6.000000,1.508727e+06,4.000000,9.000000


**Extract only numeric data **

In [0]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
Provider_Info_numeric = Provider_Info.select_dtypes(include=numerics)
Provider_Info_numeric = Provider_Info_numeric.drop(columns=['ZIP', 'PHONE', 'COUNTY_SSA'])
Provider_Info_numeric.head()

,BEDCERT,RESTOT,OVERALL_RATING,SURVEY_RATING,QUALITY_RATING,STAFFING_RATING,RN_STAFFING_RATING,AIDHRD,VOCHRD,RNHRD,...,ADJ_AIDE,ADJ_LPN,ADJ_RN,ADJ_TOTAL,INCIDENT_CNT,CMPLNT_CNT,FINE_CNT,FINE_TOT,PAYDEN_CNT,TOT_PENLTY_CNT
0,57.0,51.5,5.0,5.0,5.0,4.0,4.0,3.43572,1.16495,0.66830,...,3.11741,1.24750,0.83853,5.13047,0.0,0.0,0.0,0.0,0.0,0.0
1,85.0,74.2,3.0,3.0,5.0,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,15259.0,1.0,2.0
2,50.0,NaN,1.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,92.0,79.8,2.0,2.0,4.0,3.0,3.0,2.32722,0.82104,0.52177,...,2.40074,0.86962,0.56463,3.83026,0.0,1.0,0.0,0.0,0.0,0.0
4,103.0,98.1,3.0,3.0,4.0,3.0,2.0,2.33617,0.92407,0.23493,...,2.55126,1.08955,0.30360,3.95709,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
Provider_Info_numeric.describe()

,BEDCERT,RESTOT,OVERALL_RATING,SURVEY_RATING,QUALITY_RATING,STAFFING_RATING,RN_STAFFING_RATING,AIDHRD,VOCHRD,RNHRD,...,ADJ_AIDE,ADJ_LPN,ADJ_RN,ADJ_TOTAL,INCIDENT_CNT,CMPLNT_CNT,FINE_CNT,FINE_TOT,PAYDEN_CNT,TOT_PENLTY_CNT
count,15616.000000,15105.000000,15436.000000,15436.000000,15403.000000,15268.000000,15268.000000,14670.000000,14670.000000,14670.000000,...,14560.000000,14560.000000,14560.000000,14560.000000,15616.000000,15616.000000,15616.000000,1.561600e+04,15616.000000,15616.000000
mean,106.119813,86.558782,3.332923,2.818412,3.981952,3.065431,3.264344,2.296539,0.880983,0.672257,...,2.303975,0.880558,0.679904,3.855575,1.194032,4.198258,0.430712,1.458445e+04,0.098105,0.528817
std,60.789892,52.999889,1.395167,1.289865,1.228714,1.261812,1.337252,0.546855,0.369252,0.517324,...,0.559962,0.341518,0.441747,0.858477,3.093278,7.384479,0.773936,5.500255e+04,0.355164,0.966987
min,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.230690,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,64.000000,51.700000,2.000000,2.000000,3.000000,2.000000,2.000000,1.939590,0.663755,0.388143,...,1.931877,0.672057,0.398147,3.312228,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,99.000000,78.700000,4.000000,3.000000,4.000000,3.000000,3.000000,2.214220,0.872270,0.569555,...,2.214275,0.875080,0.587110,3.726035,0.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,127.000000,108.700000,5.000000,4.000000,5.000000,4.000000,4.000000,2.571783,1.069448,0.808613,...,2.582335,1.074725,0.854212,4.229495,1.000000,5.000000,1.000000,5.336250e+03,0.000000,1.000000
max,1389.000000,754.900000,5.000000,5.000000,5.000000,5.000000,5.000000,6.512810,5.053530,8.492540,...,6.593350,5.560940,7.775820,11.726110,126.000000,135.000000,6.000000,1.508727e+06,4.000000,9.000000


**Remove NaN.**

In [0]:
Provider_Info_numeric.replace(['NaN', 'NaT'], np.nan, inplace = True)
Provider_Info_numeric_cleaned = Provider_Info_numeric.dropna(how='any', axis = 0)
Provider_Info_numeric_cleaned.describe()

,BEDCERT,RESTOT,OVERALL_RATING,SURVEY_RATING,QUALITY_RATING,STAFFING_RATING,RN_STAFFING_RATING,AIDHRD,VOCHRD,RNHRD,...,ADJ_AIDE,ADJ_LPN,ADJ_RN,ADJ_TOTAL,INCIDENT_CNT,CMPLNT_CNT,FINE_CNT,FINE_TOT,PAYDEN_CNT,TOT_PENLTY_CNT
count,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,...,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,14557.000000,1.455700e+04,14557.000000,14557.000000
mean,107.454214,87.349440,3.379199,2.818369,3.999107,3.165487,3.374116,2.294152,0.878230,0.668123,...,2.303709,0.880563,0.679592,3.854849,1.192279,4.272240,0.436835,1.481532e+04,0.095075,0.531909
std,60.527961,52.785727,1.384830,1.285816,1.215313,1.204848,1.270249,0.544454,0.364889,0.511148,...,0.559683,0.341374,0.441242,0.856764,3.088384,7.449123,0.778117,5.581743e+04,0.347173,0.964413
min,6.000000,2.400000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.230690,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,66.000000,52.500000,2.000000,2.000000,3.000000,2.000000,3.000000,1.938830,0.663340,0.387520,...,1.931870,0.672070,0.398140,3.312220,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,100.000000,79.500000,4.000000,3.000000,4.000000,3.000000,3.000000,2.212750,0.870550,0.567820,...,2.214240,0.875090,0.587010,3.725320,0.000000,2.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,128.000000,109.100000,5.000000,4.000000,5.000000,4.000000,4.000000,2.570280,1.066380,0.805670,...,2.582100,1.074720,0.854140,4.229300,1.000000,5.000000,1.000000,5.866000e+03,0.000000,1.000000
max,1389.000000,754.900000,5.000000,5.000000,5.000000,5.000000,5.000000,6.512810,5.053530,8.492540,...,6.593350,5.560940,7.775820,11.726110,126.000000,135.000000,6.000000,1.508727e+06,4.000000,9.000000


**Move OVERALL_RATING to the first position.**

In [0]:
cols = Provider_Info_numeric_cleaned.columns.tolist()
cols = [cols[3]] + cols[:3] + cols[4:]
Provider_Info_numeric_cleaned = Provider_Info_numeric_cleaned[cols]
Provider_Info_numeric_cleaned.head()

,OVERALL_RATING,BEDCERT,SURVEY_RATING,RESTOT,QUALITY_RATING,STAFFING_RATING,RN_STAFFING_RATING,AIDHRD,VOCHRD,RNHRD,...,ADJ_AIDE,ADJ_LPN,ADJ_RN,ADJ_TOTAL,INCIDENT_CNT,CMPLNT_CNT,FINE_CNT,FINE_TOT,PAYDEN_CNT,TOT_PENLTY_CNT
0,5.0,57.0,5.0,51.5,5.0,4.0,4.0,3.43572,1.16495,0.66830,...,3.11741,1.24750,0.83853,5.13047,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,92.0,2.0,79.8,4.0,3.0,3.0,2.32722,0.82104,0.52177,...,2.40074,0.86962,0.56463,3.83026,0.0,1.0,0.0,0.0,0.0,0.0
4,3.0,103.0,3.0,98.1,4.0,3.0,2.0,2.33617,0.92407,0.23493,...,2.55126,1.08955,0.30360,3.95709,0.0,0.0,0.0,0.0,0.0,0.0
5,5.0,149.0,3.0,119.7,5.0,4.0,3.0,2.57869,1.01443,0.41055,...,2.56783,1.04823,0.46444,4.07866,0.0,1.0,0.0,0.0,0.0,0.0
6,5.0,124.0,4.0,96.0,5.0,3.0,4.0,1.99985,0.62768,0.59598,...,2.12102,0.70311,0.75448,3.52979,1.0,1.0,0.0,0.0,0.0,0.0


**Split Train and Test data.**

In [0]:
x_train, x_test, y_train, y_test = train_test_split(Provider_Info_numeric_cleaned.iloc[:,1:], Provider_Info_numeric_cleaned.iloc[:,0], test_size = .2, random_state = 43)

In [0]:
x_train.head()

,BEDCERT,SURVEY_RATING,RESTOT,QUALITY_RATING,STAFFING_RATING,RN_STAFFING_RATING,AIDHRD,VOCHRD,RNHRD,TOTLICHRD,...,ADJ_AIDE,ADJ_LPN,ADJ_RN,ADJ_TOTAL,INCIDENT_CNT,CMPLNT_CNT,FINE_CNT,FINE_TOT,PAYDEN_CNT,TOT_PENLTY_CNT
13487,54.0,2.0,31.2,4.0,5.0,5.0,2.95350,1.45921,1.59812,3.05733,...,2.67316,1.23426,1.25358,5.25936,10.0,9.0,2.0,120369.0,1.0,3.0
7149,40.0,3.0,38.4,5.0,1.0,2.0,1.00483,0.57287,0.23909,0.81196,...,1.50190,0.79959,0.37556,2.68619,0.0,0.0,0.0,0.0,0.0,0.0
1015,121.0,2.0,118.2,5.0,2.0,2.0,2.07439,0.79683,0.32475,1.12159,...,2.03111,0.84839,0.38146,3.25391,15.0,15.0,0.0,0.0,0.0,0.0
2585,50.0,4.0,46.8,3.0,2.0,2.0,1.73202,1.23724,0.31842,1.55566,...,1.77061,1.31478,0.35352,3.42313,0.0,0.0,1.0,15265.0,0.0,1.0
2051,120.0,3.0,114.0,5.0,4.0,4.0,2.56633,0.97588,0.57167,1.54755,...,2.32180,0.97664,0.60328,3.87363,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
x_train.describe()

,BEDCERT,SURVEY_RATING,RESTOT,QUALITY_RATING,STAFFING_RATING,RN_STAFFING_RATING,AIDHRD,VOCHRD,RNHRD,TOTLICHRD,...,ADJ_AIDE,ADJ_LPN,ADJ_RN,ADJ_TOTAL,INCIDENT_CNT,CMPLNT_CNT,FINE_CNT,FINE_TOT,PAYDEN_CNT,TOT_PENLTY_CNT
count,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,...,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,11645.000000,1.164500e+04,11645.000000,11645.000000
mean,107.416917,2.826106,87.375380,4.003950,3.169257,3.379648,2.293850,0.875126,0.666240,1.541366,...,2.303370,0.877879,0.678369,3.850014,1.205754,4.271275,0.435380,1.482966e+04,0.093860,0.529240
std,59.947987,1.286469,51.915365,1.210775,1.197565,1.264578,0.544839,0.364225,0.500204,0.566864,...,0.560402,0.341884,0.432729,0.852740,2.970700,7.497108,0.779582,5.720135e+04,0.344811,0.965094
min,6.000000,1.000000,2.400000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.085770,...,0.000000,0.000000,0.000000,1.448440,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,66.000000,2.000000,52.400000,3.000000,2.000000,3.000000,1.939090,0.660640,0.388700,1.258320,...,1.931870,0.669700,0.399570,3.314370,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,100.000000,3.000000,79.500000,4.000000,3.000000,3.000000,2.213000,0.867720,0.566920,1.445610,...,2.214790,0.872790,0.586160,3.723900,0.000000,2.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,128.000000,4.000000,109.300000,5.000000,4.000000,4.000000,2.568490,1.063180,0.805010,1.681870,...,2.579010,1.072220,0.855520,4.222850,1.000000,5.000000,1.000000,5.807000e+03,0.000000,1.000000
max,1389.000000,5.000000,754.900000,5.000000,5.000000,5.000000,6.512810,5.053530,8.492540,8.492540,...,6.593350,5.560940,7.775820,11.726110,72.000000,135.000000,6.000000,1.508727e+06,4.000000,9.000000


In [0]:
y_train.describe()

count    11645.000000
mean         3.388665
std          1.382489
min          1.000000
25%          2.000000
50%          4.000000
75%          5.000000
max          5.000000
Name: OVERALL_RATING, dtype: float64

**Scale Data.**

In [0]:
scaler_x = StandardScaler()
scaler_x.fit(x_train)
x_train_scaled = scaler_x.transform(x_train)
x_test_scaled = scaler_x.transform(x_test)

In [0]:
x_train_scaled.view()

array([[-0.89109265, -0.64217698, -1.08210338, ...,  1.84512921,
         2.6280433 ,  2.56023391],
       [-1.12463846,  0.13517759, -0.94341016, ..., -0.2592648 ,
        -0.27221866, -0.54840545],
       [ 0.22659086, -0.64217698,  0.59377303, ..., -0.2592648 ,
        -0.27221866, -0.54840545],
       ...,
       [-0.29054629,  0.13517759, -0.10547196, ..., -0.17265473,
        -0.27221866,  0.48780767],
       [ 0.20990902,  0.91253215, -0.2711333 , ..., -0.2592648 ,
        -0.27221866, -0.54840545],
       [ 0.1264998 , -1.41953155, -0.48109943, ..., -0.2592648 ,
         2.6280433 ,  0.48780767]])

## Dimension Reduction with PCA

In [0]:
pca_two = PCA(n_components=2)
x_train_pca_two = pca_two.fit_transform(x_train_scaled)
x_test_pca_two = pca_two.transform(x_test_scaled)

pca_sixteen = PCA(n_components=16)
x_train_pca_sixteen = pca_sixteen.fit_transform(x_train_scaled)
x_test_pca_sixteen = pca_sixteen.transform(x_test_scaled)

In [0]:
x_test_pca_two

array([[ 3.912674  ,  2.01051568],
       [-1.77958784, -1.41805599],
       [-2.30399105,  0.37615466],
       ...,
       [ 0.30505432, -0.73647251],
       [-2.40307564, -2.47563107],
       [-0.32391314,  0.10533594]])

## Model Generation

### Model #1 Logistic Regression with original data


In [0]:
log_reg1 = LogisticRegression(solver='lbfgs', multi_class='multinomial')
log_reg1.fit(x_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

**Calculate predicted output.**

In [0]:
y_predict_train_1 = log_reg1.predict(x_train_scaled)
y_predict_test_1 = log_reg1.predict(x_test_scaled)

**Mean Accuracy Score**

In [0]:
print(log_reg1.score(x_train_scaled, y_train))
print(log_reg1.score(x_test_scaled, y_test))

0.8535852297123229
0.8475274725274725


**Confusion Matrix**

In [0]:
confusion_matrix(y_train, y_predict_train_1)

array([[1048,  290,    0,    0,    0],
       [ 235, 1795,  283,    0,    0],
       [   0,  263, 1366,  267,    0],
       [   0,    0,   90, 2407,  184],
       [   0,    0,    0,   93, 3324]])

In [0]:
confusion_matrix(y_test, y_predict_test_1)

array([[286,  75,   0,   0,   0],
       [ 72, 453,  60,   0,   0],
       [  0,  66, 356,  75,   0],
       [  0,   0,  27, 571,  39],
       [  0,   0,   0,  30, 802]])

### Model #2 Logistic Regression with PCA (n=2)

In [0]:
log_reg2 = LogisticRegression(solver='lbfgs', multi_class='multinomial')
log_reg2.fit(x_train_pca_two, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

**Calculate predicted output**

In [0]:
y_predict_train_2 = log_reg2.predict(x_train_pca_two)
y_predict_test_2 = log_reg2.predict(x_test_pca_two)

**Mean Accuracy Score**

In [0]:
print(log_reg2.score(x_train_pca_two, y_train))
print(log_reg2.score(x_test_pca_two, y_test))

0.38608844997853153
0.38461538461538464


**Confusion Matrix**

In [0]:
confusion_matrix(y_train, y_predict_train_2)

array([[140, 179,   0,  20,  22],
       [111, 255,   0,  53, 166],
       [ 41, 224,   0,  53, 179],
       [ 22, 128,   0,  82, 405],
       [ 12, 121,   0,  56, 643]])

In [0]:
confusion_matrix(y_test, y_predict_test_2)

array([[140, 179,   0,  20,  22],
       [111, 255,   0,  53, 166],
       [ 41, 224,   0,  53, 179],
       [ 22, 128,   0,  82, 405],
       [ 12, 121,   0,  56, 643]])

### Model #3 Logistic Regression with PCA (n=16)

In [0]:
log_reg3 = LogisticRegression(solver='lbfgs', multi_class='multinomial')
log_reg3.fit(x_train_pca_sixteen, y_train)

**Calculate predicted output**

In [0]:
y_predict_train_3 = log_reg3.predict(x_train_pca_sixteen)
y_predict_test_3 = log_reg3.predict(x_test_pca_sixteen)

**Mean Accuracy Score**

In [0]:
print(log_reg3.score(x_train_pca_sixteen, y_train))
print(log_reg3.score(x_test_pca_sixteen, y_test))

0.8437097466723916
0.8379120879120879


**Confusion Matrix**

In [0]:
confusion_matrix(y_train, y_predict_train_3)

array([[1040,  298,    0,    0,    0],
       [ 240, 1807,  266,    0,    0],
       [   0,  235, 1410,  251,    0],
       [   0,    1,  128, 2331,  221],
       [   0,    0,    0,  180, 3237]])

In [0]:
confusion_matrix(y_test, y_predict_test_3)

array([[283,  78,   0,   0,   0],
       [ 73, 458,  54,   0,   0],
       [  0,  60, 364,  73,   0],
       [  0,   1,  36, 555,  45],
       [  0,   0,   0,  52, 780]])

### Model Comparison

In the first model, which is a full model with 26 features, the accuracy score comes out to be approximately 0.85 for both train and test data. 

The second model uses only two principal components from PCA, and the accuracy score is much worse (0.38). This is expected as the number of features are reduced considerably compared to the full model. 

In the third model, however, it can be observed that the accuracy score is comparable to the full model (0.84) even though the number of features are still much less than that of the full model (n=16 vs n=26). This demonstrates that the 16 principal components used in the third model is able to explain most of the original 26 features.


In [0]:
!jupyter nbconvert --to html '/content/gdrive/My Drive/Colab Notebooks/MLAssignment3.ipynb'

[NbConvertApp] Converting notebook /content/gdrive/My Drive/Colab Notebooks/MLAssignment3.ipynb to html
[NbConvertApp] Writing 351799 bytes to /content/gdrive/My Drive/Colab Notebooks/MLAssignment3.html
